In [2]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so

import warnings
warnings.filterwarnings('ignore')

from scipy import stats
import pingouin as pg
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Chap 13. 구조방정식 모델링
## 13.1 구조방정식 모델링 개요
- 직접적인 측정이 어려운 잠재변수 간의 영향 관계를 분석하기 위한 다변량 통계분석
- 관측변수를 이용하여 잠재요인을 간접적으로 측정한 후, 잠재요인 간의 이론적 영향관계 분석
=> 요인분석(confirmatory factor analysis) + 회귀분석
- 측정모델(관측변수와 잠재변수 간의 관계) + 구조모델(잠재변수 간의 관계)
    - 외생잠재변수 : 모델 내의 다른 잠재변수에 영향을 미치는 변수, 독립변수 역할 수행
    - 외생관측변수 : 외생잠재변수 측정 (x)
    - 내생잠재변수 : 모델 내의 외생잠재변수에 의해 직/간접적으로 영향을 받는 변수, 모델 내에서 독립/종속변수 역할
    - 내생관측변수 : 내생잠재변수 측정 (y)
- 공분산행렬이 관심값, 개별 케이스가 아닌 개별 케이스로부터 얻어진 공뷴산행렬에 초점
=> 모델에 의해 예측되는 잠재변수와 관측변수간의 공분산행렬이 실제 표본의 공분산행렬과 유사한 경우, 모델이 실제 데이터를 잘 반영한다고 추정
- 구조갯수, 요인적재값, 공분산, 구조오차, 측정오차 등
- 구조갯수 : 잠재변수간의 경로갯수, 가설 갯수
- 측정오차 : 잠재변수가 관측변수를 완벽하게 설명하지 못하는 정도 
- 구조오차 : 독립변수로서 내생잠재변수와 외생잠재변수에 의해 설명되지 않고 남아있는 내생잠재변수의 오차

## 13.2 구조방정식 모델링 분석 절차
### 13.2.1 측정모델 : 잠재변수 및 관측변수의 단일차원성, 신뢰도, 타당도 분석 & 평가
- 확인적 요인분석 : 모든 잠재변수에 대해서 수행, 잠재변수 별 개별 평가도 가능하나 일반적으로 전체 잠재변수가 하나의 모델로 구성된 통합측정모델 대상으로 한꺼번에 측정
    - 단일차원성 (unidimensionality) : 측정모델을 구성하는 관측변수는 오직 하나의 구성개념(잠재변수) 측정
    1) 적합도 (fitness) : 각 범주 별로 적어도 한 개의 지표는 사용할 것
        - 절대적합도 : Chisq (p-val > 0.05, 관측된 공분산행렬과 모델에 의해 예측된 공분산행렬이 같다는 귀무가설 검정 / chi2는 표본크기(200기준) 증가에 따라 함께 커지는 경향이 있어 쉽게 기각됨), RMSEA (<0.08), GFI (>0.9), AGFI (>0.9)
        - 중분적합도 : CFI (>0.9), TLI (>0.9), NFI (>0.9)
        - 간명적합도 : Chisq/df (<3)
    2) 요인적재값 (factor loading, $\geq 0.6$) : 
        - 단일 차원성은 각 잠재변수에 대해 모든 관측변수가 적절 수준 이상(0.6)의 요인적재값을 가질 때 충족
        - 충족하지 못할 경우, 낮은 요인적재값을 한번에 한 개씩 제거하고 적정 수준 도달시 까지 반복
    3) 수정지표 (modification index)
        - 수정지표 : 추정할 새로운 모수를 모델에 추가할 때 변화하는 적합도 크기(설명력이 증가하여, 감소하는 $\chi^2$값)
        - 모델에 새로운 모수를 도입함으로써 데이터를 보다 잘 설명하는 좀 더 복잡한 형태의 모델 생성
            => 충분한 이론적 뒷받침 필요
        - 혹은 모델의 적합도에 기여하지 못하는 기존의 불필요한 모수를 제거함으로써 좀더 간명한 모델 생성
    - 신뢰도 (reliability) : 측정척도가 측정하려고 의도하는 것을 얼마나 정확하게 오차없이 측정하고 있는지
    1) 크론바흐 알파계수 (cronbach's coefficient $\alpha$, $\geq 0.7$)
    2) 복합신뢰도 (composite reliability CR, $\geq 0.7$)
        $$CR = \frac{(\sum^n_1\lambda)^2}{(\sum^n_1\lambda)^2 + \sum^n_1\delta(\epsilon)}=\frac{(\sum^n_1\lambda)^2}{(\sum^n_1\lambda)^2+\sum^n_1(1-\lambda^2)}$$
    3) 평균분산추출 (average variance extracted AVE, $\geq 0.6$)
    $$AVE=\frac{\sum^n_1\lambda^2}{\sum^n_1\lambda^2+\sum^n_1\delta(\epsilon)}=\frac{\sum^n_1\lambda^2}{\sum^n_1\lambda^2+\sum^n_1(1-\lambda^2)}=\frac{\sum^n_1\lambda^2}{n} =\frac{잠재변수로\ 설명될\ 수\ 있는\ 분산의\ 크기}{n}=잠재변수에\ 대한\ 관측변수의\ 평균적\ 설명력 \\ \lambda:표준화\ 요인적재값,\ \delta,\ \epsilon:측정오차,\ n:측정변수\ 개수$$
    - 타당도 (validity) : 측정측도가 측정하려고 의도하는 것을 얼마나 충실하게 측저앟고 있는지
    1) 집중타당도 (convergent validity) : 측정척도가 측정하기로 되어있는 잠재변수와의 관련정도
        - 요인적재값 (모든 변수값이 통계적으로 유의) : 클수록 유의
        - 평균분산추축 (AVE, $\geq$0.5) : 하나의 요인적재값이 낮은 변수가 크게 낮춤
    2) 판별타당도 (discriminant validity) : 측정척도가 측정하지 않기로 되어있는 다른 잠재변수와는 관련 갖지 않는 정도
        - $\sqrt{평균분산추출}$ ($\sqrt{ }$average variance extracted AVE, > 상관계수)
### 13.2.2 구조모델 : 검증된 관측변수 기반 구조모델 바탕으로 잠재변수 간 경로분석 & 영향관계(가설) 검증
- 잠재변수 간의 관계에 초점. 연구모델에 의해 설정된 이론적 관계가 데이터에 의해 지지되는지 검토
1) 모델 적합도 평가
2) 가설 검정 : p-val < 0.05, 경로계수의 부호는 가설화 된 관계의 방향과 일치
3) 독립변수 설명력 평가 : 결정계수($R^2$)


## 13.3 측정모델/구조모델

In [4]:
# y1-y4 : 1960 민주화 수준 / y5-y8 : 1965 민주화 수준 / x1-x3 : 1960 산업화 수준
# < 1965 민주화 수준은, 1960 민주화 수준과 산업화 수준에 의해 결정되는가? >
# 1960 산업화 수준 : 외생잠재변수, 모델 외부요인에 의해 설명 가정
# 1960, 1965 민주화 수준 : 내생잠재변수, 모델 내부 요인에 의해 결정 가정
bollen = pd.read_csv('../data/kwak/Bollen.csv').drop('rownames', axis=1)
bollen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   y1      75 non-null     float64
 1   y2      75 non-null     float64
 2   y3      75 non-null     float64
 3   y4      75 non-null     float64
 4   y5      75 non-null     float64
 5   y6      75 non-null     float64
 6   y7      75 non-null     float64
 7   y8      75 non-null     float64
 8   x1      75 non-null     float64
 9   x2      75 non-null     float64
 10  x3      75 non-null     float64
dtypes: float64(11)
memory usage: 6.6 KB
